In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import os
import json
import random
    
# load source words
source_words_path = os.path.join(os.getcwd(), 'source_words.pkl')
with open(source_words_path, 'rb') as f_source_words:
    source_words = pickle.load(f_source_words)
    
# load target words
target_words_path = os.path.join(os.getcwd(), 'target_words.pkl')
with open(target_words_path, 'rb') as f_target_words:
    target_words = pickle.load(f_target_words)
    
# load label words
label_words_path = os.path.join(os.getcwd(), 'label_words.pkl')
with open(label_words_path, 'rb') as f_label_words:
    label_words = pickle.load(f_label_words)

In [2]:
print(len(source_words))
print(len(target_words))
print(len(label_words))
print(source_words['<pad>'])

945
133
27
1


In [3]:
'''
以下注意这个模型中encoder与decoder都使用n_layers=2所以在计算attention时，拿到上一步hidden的最后一层是hidden[-1,:,:]
'''
# 构建编码器
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, dropout, pad_index):
        super(Encoder, self).__init__()
        self.pad_index = pad_index
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_index)
        self.gru = nn.GRU(emb_dim, hidden_dim, n_layers, dropout=dropout, bidirectional=True, batch_first=True) #使用双向
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
    def forward(self, src, src_len):
        # 初始化
        # h0 = torch.zeros(self.n_layers, src.size(1), self.hidden_dim).to(device)
        # c0 = torch.zeros(self.n_layers, src.size(1), self.hidden_dim).to(device)
        # nn.init.kaiming_normal_(h0)
        # nn.init.kaiming_normal_(c0)
        # src=[batch_size, seq_len]
        embedded = self.dropout(self.embedding(src))
        # embedd=[batch_size,seq_len,embdim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, src_len, batch_first=True, enforce_sorted=True) #这里enfore_sotred=True要求数据根据词数排序
        output, hidden = self.gru(packed)
        # output=[batch_size, seq_len, hidden_size*2]
        # hidden=[n_layers*2, batch_size, hidden_size]
        
        output, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True, padding_value=self.pad_index, total_length=len(src[0])) #这个会返回output以及压缩后的legnths
        
        '''
        hidden[-2,:,:]是gru最后一步的forward
        hidden[-1,:,:]是gru最后一步的backward
        利用最后前向和后向的hidden的隐状态作为decoder的初始状态
        hidden:[batch_size, hidden_dim]
        '''
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        return output, hidden

# 构建attention权重计算方式
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear((hidden_dim * 2) + hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def concat_score(self, hidden, encoder_output):
        seq_len = encoder_output.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1) # [batch_size, seq_len, hidden_size]
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_output),dim=2))) # [batch_size, seq_len, hidden_dim]
        attention = self.v(energy).squeeze(2) #[batch_size, seq_len]
        return attention #[batch_size, seq_len]

    def forward(self, hidden, encoder_output):
        # hidden = [batch_size, hidden_size]
        # #encoder_output=[batch_size, seq_len, hidden_dim*2]
        
        attn_energies = self.concat_score(hidden, encoder_output)

        return F.softmax(attn_energies, dim=1).unsqueeze(1) #softmax归一化，[batch_size, 1, seq_len]

# 构建解码器
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, dropout):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU((hidden_dim * 2) + emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        # 槽填充slot filling
        self.slot_out = nn.Linear(hidden_dim * 2 + hidden_dim, output_dim)
        self.attention = Attention(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_output):
        input = input.unsqueeze(1)
        # input=[batch_size, 1]
        # hidden=[batch_size, hidden_size] 初始化为encoder的最后一层 [batch_size, hidden_size]
        # encoder_output=[batch_size, seq_len, hidden_dim*2]
        
        # embedded=[batch_sze, 1, emb_dim]
        embedded = self.dropout(self.embedding(input))

        # 利用利用上一步的hidden与encoder_output，计算attention权重
        # attention_weights=[batch_size, 1, seq_len]
        attention_weights = self.attention(hidden, encoder_output)

        '''
        以下是计算上下文：利用attention权重与encoder_output计算attention上下文向量
        注意力权重分布用于产生编码器隐藏状态的加权和，加权平均的过程。得到的向量称为上下文向量
        '''
        context = attention_weights.bmm(encoder_output) # [batch_size, 1, seq_len]*[batch_size,seq_len,hidden_dim*2]=[batch_size, 1, hidden_dim*2]
        
        #拼接注意力上下文和embedding向量作为gru输入
        # [batch_size, 1, hidden_dim*2+emb_dim]
        gru_input = torch.cat([context, embedded], 2)
        
        # 将注意力向量，本次embedding以及上次的hidden输入到ｇｒｕ中
        # decoder_output=[batch_size, seq_len, hidden_size]
        # hidden=[n_layers, batch_size, hidden_size]
        # decoder中的ｇｒｕ是单向，序列长度为１，层为１，
        # 所以decoder_output=[batch_size, １, hidden_size]，hidden=[１, batch_size, hidden_size]
        decoder_output, hidden = self.gru(gru_input, hidden.unsqueeze(0))
        

        decoder_output_context = torch.cat([decoder_output, context], 2) # 连接context与decoder_output的hidden_dim =[batch_size, 1, 2 * hidden_dim + hidden_dim]
        prediction = self.slot_out(decoder_output_context.squeeze(1))
        # prediction=[batch_size, output_dim]，词汇表中所有词的概率分布，这里可以使用softmax进行归一化
        return prediction, hidden.squeeze(0), attention_weights.squeeze(1), context.squeeze(1)

# 利用Encoder与Decoder构建seq2seq模型
class Seq2Seq(nn.Module):
    '''
    接收source句子
    利用编码器encoder生成上下文向量
    利用解码器decoder生成预测target句子

    每次迭代中：
    传入input以及先前的hidden与cell状态给解码器decoder
    从解码器decoder中接收一个prediction以及下一个hidden与下一个cell状态
    保存这个prediction作为预测句子中的一部分
    决定是否使用"teacher force":
        如果使用：解码器的下一次input是真实的token
        如果不使用：解码器的下一次input是预测prediction（使用output tensor的argmax）的token
    '''

    def __init__(self, predict_flag, encoder, decoder, intent_size):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.predict_flag = predict_flag
        # 意图分类
        self.intent_out = nn.Linear((encoder.hidden_dim * 2) + encoder.hidden_dim, intent_size)
        assert encoder.hidden_dim == decoder.hidden_dim, 'encoder与decoder的隐藏状态维度必须相等！'
        assert encoder.n_layers == decoder.n_layers, 'encoder与decoder的层数必须相等！'
        
    def forward(self, src, src_lens, trg, teacher_forcing_ration=1.0):
        '''
        src=[batch_size, seq_len]
        src_len=[batch_size]
        trg=[batch_size, trg_len]
        
        '''
        # 预测，一次输入一句话
        if self.predict_flag:
            assert len(src) == 1, '预测时一次输入一句话'
            src_len = len(src[0])
            output_tokens = []
            encoder_output, encoder_hidden = self.encoder(src, src_lens)
            hidden = encoder_hidden
            input = torch.tensor(2).unsqueeze(0)  # 预测阶段解码器输入第一个token-> <sos>
            for s in range(1, src_len):
                if s == 1:
                    # context = [batch_size, hidden_dim*2]
                    output, hidden, _, context = self.decoder(input, hidden, encoder_output)
                else:
                    output, hidden, _, _ = self.decoder(input, hidden, encoder_output)
                    
                input = output.argmax(1)
                output_token = input.squeeze().detach().item()
               
                output_tokens.append(output_token)
            concated = torch.cat((encoder_hidden, context), 1)
            intent_outputs = self.intent_out(concated)
            intent_outputs = intent_outputs.squeeze()
            intent_outputs = intent_outputs.argmax()
            return output_tokens, intent_outputs

        # 训练
        else:
            '''
            src=[batch_size, seq_len]
            trg=[batch_size, trg_len]
            teacher_forcing_ration是使用teacher forcing的概率,例如teacher_forcing_ration=0.8，则输入的时间步有80%的真实值。
            '''
            batch_size = trg.shape[0]
            trg_len = trg.shape[1]
            trg_vocab_size = self.decoder.output_dim
            # 存储decoder outputs
            slot_outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(device)
            # encoder的最后一层hidden state(前向＋后向)作为decoder的初始隐状态,[batch_size, seq_len, hidden_size*2]
            # hidden=[batch_size, hidden_size]
            encoder_output, encoder_hidden = self.encoder(
                src, src_lens)  
            hidden = encoder_hidden
            
            # 输入到decoder的第一个是<sos>
            input = trg[:, 0] # [batch_size]
            
            for t in range(1, trg_len):
                '''
                解码器输入的初始hidden为encoder的最后一步的hidden
                接收输出即predictions和新的hidden状态
                '''
                if t == 1:
                    # context = [batch_size, hidden_dim*2]
                    output, hidden, _, context = self.decoder(input, hidden, encoder_output)
                else:
                    output, hidden, _, _ = self.decoder(input, hidden, encoder_output)
                # 存入decoder的预测值
                slot_outputs[:, t, :] = output
                # 是否使用teacher forcing
                teacher_force = random.random() < teacher_forcing_ration
                # 获取预测的最大概率的token
                predict_max = output.argmax(1)
                '''
                如果是teacher forcing则下一步使用真实token作为解码的输入
                否则使用decoder的预测值作为下一步的解码输入
                '''
                input = trg[:, t] if teacher_force else predict_max
            # concated = [batch_size, hidden_dim * 2 + hidden_dim]
            concated = torch.cat((encoder_hidden, context), 1)
            intent_outputs = self.intent_out(concated)
            # slot_outputs=[batch_size, trg_len, trg_vocab_size], intetn_outputs=[batch_size, intent_size]
            return slot_outputs, intent_outputs

In [6]:
encoder_embedding_dim = 64
decoder_embedding_dim = 64
hidden_dim = 128
n_layers = 1
encoder_dropout = 0.1
decoder_dropout = 0.1

model_path = os.path.join(os.getcwd(), "model.h5")

input_dim = len(source_words) # source 词典大小（即词数量）
output_dim = len(target_words) # target 词典大小（即实体类型数量）
label_dim = len(label_words) # label 词典大小（即意图类别数量）

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim, n_layers, encoder_dropout, source_words['<pad>'])
decoder = Decoder(output_dim, decoder_embedding_dim, hidden_dim, n_layers, decoder_dropout)

model = Seq2Seq(True, encoder, decoder, label_dim)

model.load_state_dict(torch.load(model_path))
model.eval()

sentence = 'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis'
with torch.no_grad():
    tokenized = list(sentence)  # tokenize the sentence
    tokenized.append('<eos>')
    indexed = [source_words[t] for t in tokenized]  # convert to integer sequence
    tensor = torch.LongTensor(indexed)  # convert to tensor
    tensor = tensor.unsqueeze(0)  # reshape in form of batch,no. of words

    slot_outputs, intent_outputs = model(tensor, [len(tensor)], None)  # prediction
    intent = intent_outputs.detach().item()
    slot_prediction = [target_words.itos[t] for t in slot_outputs]

    print('slot_prediciton:{}'.format(' '.join(slot_prediction)))
    print('intent_prediction:{}'.format(label_words.itos[intent]))

slot_prediciton:o o o o o o o o o <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
intent_prediction:flight


D:\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
